In [ ]:
#default_exp utils

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Utils

In [ ]:
#export
import random
from itertools import chain
from math import ceil, log10

import numpy as np
import pandas as pd


In [ ]:
#export
def generate_daily_series(n_series: int, 
                          min_length: int = 50, 
                          max_length: int = 500,
                          n_static_features: int = 0,
                          equal_ends: bool = False,
                          seed: int = 0):
    """Generates `n_series` of different lengths in the interval [`min_length`, `max_length`].
    
    If `n_static_features > 0`, then each serie gets static features with random values.
    If `equal_ends == True` then all series end at the same date."""
    rng = np.random.RandomState(seed)
    random.seed(seed)
    series_lengths = rng.randint(min_length, max_length+1, n_series)
    total_length = series_lengths.sum()
    n_digits = ceil(log10(n_series))
    
    dates = pd.date_range('2000-01-01', periods=max_length, freq='D').values
    uids = [[f'id_{i:0{n_digits}}'] * serie_length for i, serie_length in enumerate(series_lengths)]
    if equal_ends:
        ds = [dates[-serie_length:] for serie_length in series_lengths]
    else:
        ds = [dates[:serie_length] for serie_length in series_lengths]
    y = np.arange(total_length) % 7 + rng.rand(total_length) * 0.5
    series = pd.DataFrame({
        'unique_id': list(chain.from_iterable(uids)),
        'ds': list(chain.from_iterable(ds)),
        'y': y
    })
    for i in range(n_static_features):
        static_values = [[random.randint(0, 100)] * serie_length for serie_length in series_lengths]
        series[f'static_{i}'] = list(chain.from_iterable(static_values))
        series[f'static_{i}'] = series[f'static_{i}'].astype('category')
        if i == 0:
            series['y'] = series['y'] * (1 + series[f'static_{i}'].cat.codes)
    series['unique_id'] = series['unique_id'].astype('category')
    series['unique_id'] = series['unique_id'].cat.as_ordered()
    series = series.set_index('unique_id')
    return series

Generate 20 series with lengths between 100 and 1,000.

In [ ]:
n_series = 20
min_length = 100
max_length = 1000

series = generate_daily_series(n_series, min_length, max_length)
series

In [ ]:
series_sizes = series.groupby('unique_id').size()
assert series_sizes.size == n_series
assert series_sizes.min() >= min_length
assert series_sizes.max() <= max_length

We can also add static features to each serie (these can be things like product_id or store_id). Only the first static feature (`static_0`) is relevant to the target.

In [ ]:
n_static_features = 2

series_with_statics = generate_daily_series(n_series, min_length, max_length, n_static_features)
series_with_statics

In [ ]:
for i in range(n_static_features):
    assert all(series_with_statics.groupby('unique_id')[f'static_{i}'].nunique() == 1)

If `equal_ends=False` (the default) then every serie has a different end date. 

In [ ]:
assert series_with_statics.groupby('unique_id')['ds'].max().nunique() > 1

We can have all of them end at the same date by specifying `equal_ends=True`.

In [ ]:
series_equal_ends = generate_daily_series(n_series, min_length, max_length, equal_ends=True)

assert series_equal_ends.groupby('unique_id')['ds'].max().nunique() == 1

In [ ]:
#export
def get_last_n_mask(serie, n):
    """Return a boolean mask where the last `n` values are True."""
    mask = np.full_like(serie, True, dtype=bool)
    n_samples = serie.shape[0]
    n_first = max(0, n_samples -  n)
    mask[:n_first] = False
    return mask

In [ ]:
np.testing.assert_equal(get_last_n_mask(pd.Series([1, 2, 3]), 2), np.array([False, True, True]))
np.testing.assert_equal(get_last_n_mask(pd.Series([1, 2, 3]), 4), np.array([True, True, True]))